In [ ]:
%config IPCompleter.greedy=True
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
# https://matplotlib.org/3.1.1/gallery/style_sheets/fivethirtyeight.html
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
Base = automap_base()

In [ ]:
# reflect an existing database into a new model
engine = create_engine("sqlite:///hawaii.sqlite")
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes
Base.classes.keys()

In [ ]:
# mapped classes are now created with names by default
# matching that of the table name.
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Inspecting the actual Table structure

In [ ]:
print(type(Measurement))
print(Measurement)

In [ ]:
print(type(Station))
print(Station)

In [ ]:
measurement_df = pd.read_sql_table('measurement', con=engine)
measurement_df

In [ ]:
station_df = pd.read_sql_table('station', con=engine)
station_df

In [ ]:
# Ease to read docs https://pysqlite.readthedocs.io/en/latest/sqlite3.html
# But I need to know the actual types in the sqlite
# I should convert this to use SQLAlchemy
import sqlite3

# Do not use con nor conn so you don't get confused with
# the conection with sqlalchemy
conn_sqlite = sqlite3.connect('hawaii.db')

# Getting the schema of SQLite3 table in Python
def sqlite_table_schema(conn, name):
    cursor = conn.execute("SELECT sql FROM sqlite_master WHERE name=?;", [name])
    sql = cursor.fetchone()[0]
    cursor.close()
    return sql
print(sqlite_table_schema(conn_sqlite, 'measurement'))
print(sqlite_table_schema(conn_sqlite, 'station'))

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the last data point in the database 
prev_year = dt.date(2017, 8, 23)
print(type(prev_year))
print(prev_year)

# Calculate the date one year from the last date in data set, so we created a function that allows us to trace back a certain number days.
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
## print(prev_year)

# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date, Measurement.prcp)
## print(results)

# To filter out all of the data that is older than a year from the last record date.
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= prev_year)
## print(results)

# Add a function that extracts all of the results from our query and put them in a list. To do this, add .all() to the end of our existing query. 
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= prev_year).all()
print(type(results))
print(type(results[0][0]))
print(results[0][0])
# print(results)

# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(results, columns=['date','precipitation'])
df.set_index(df['date'], inplace=True)
## print(df.to_string(index=False))

# Check the dataframe structure
## print(df.head())

# Sort the dataframe by date
df = df.sort_index()
## print(df.to_string(index=False))

# Use Pandas Plotting with Matplotlib to plot the data
df.plot()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

In [ ]:
# How many stations are available in this dataset?
session.query(func.count(Station.station)).all()

In [ ]:
# What are the most active stations?
# List the stations and the counts in descending order.
session.query(Measurement.station, func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
filter(Measurement.station == 'USC00519281').all()

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
results = session.query(Measurement.tobs).\
filter(Measurement.station == 'USC00519281').\
filter(Measurement.date >= prev_year).all()
## print(results)
df = pd.DataFrame(results, columns=['tobs'])
## print(df.head())

# Plot the df as Histogram
df.plot.hist(bins=12)
plt.tight_layout()

In [ ]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates


# Challenge